In [1]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = '0'
#import matplotlib
import tensorflow as tf
#import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, LSTM, GRU, Conv1D, Activation, Lambda, Permute, Conv2D, Flatten
from tensorflow.keras.models import Model, load_model
from keras.constraints import maxnorm, nonneg
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, EarlyStopping, TensorBoard
#from tensorflow.keras.utils.vis_utils import model_to_dot
from tqdm import tqdm,trange
import gc
from argparse import ArgumentParser
from itertools import product,combinations 

import pandas as pd
import sys
from keras.constraints import maxnorm, nonneg
import time

from math import comb
from sklearn.linear_model import Ridge
from numpy import linalg as LA
from sklearn.metrics import mean_squared_error
os.environ["CUDA_VISIBLE_DEVICES"]="0"

2023-05-11 11:38:08.673498: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Utils

In [2]:
def save_pd(history, fileName):
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 

    # save to json:  
    hist_json_file = fileName + '_history.json' 
    with open(hist_json_file, mode='w') as f:
        hist_df.to_json(f)

    # save to csv: 
    hist_csv_file = fileName + '_history.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)

features = np.load("./feature.npy")
all_labels = np.load("./all_labels.npy")
print("shape info:",[features.shape, features.dtype, all_labels.shape])

image_height = 28
image_width = 28

patience = 10

def get_matrix(n):
    I = np.eye(d)
    M1 = np.vstack([I]*((d-1)))
    P = np.roll(I,1,axis=-1)
    M2 = P@I
    P_ = P
    for i in range(1,d-1):
        P1 = P_@P
        M2 = np.vstack((M2,P1@I))
        P_ = P1
    return M1,M2

shape info: [(200000, 10, 784), dtype('float16'), (200000, 10)]


## Cyclic Model for image sum

In [3]:
def cyclic_model(d,l1_lambda):
    l1_reg = tf.keras.regularizers.l1(l1_lambda) 

    input_img = Input(shape=(d, 28*28)) #Layer 0
    #print('input_img:',input_img.shape)

    input_img_p = Permute((2,1))(input_img) #Layer 1

    linear1 = Dense(d,activation=None)(input_img_p) #Layer 2
    inputs1 = Dense(d*(d-1),use_bias=False,activation=None)(linear1) #Layer 3
    #print('inputs1:',inputs1.shape)
    inputs2 = Dense(d*(d-1),use_bias=False,activation=None)(linear1) #Layer 4
    #print('inputs2:',inputs2.shape)

    x = tf.concat((inputs1,inputs2), axis=1)     #Layer 5 # (B, 2*H*W, d*(d-1))
    linear2 = Dense(d*(d-1),activation=None)(x) #Layer 6
    #new_dense2 = Dense(d*(d-1),use_bias=False,activation=None)(x)
    x = Permute((2,1))(linear2) #layer 7 # (B, d*(d-1), 2*H*W)
    #print('x shape:',x.shape)
    
    x = Dense(300, activation='tanh')(x)   
    x = Dense(100, activation='tanh')(x)
    x = Dense(30, activation='tanh')(x)
    #print('x shape:',x.shape)
    
    Adder = Lambda(lambda x: K.sum(x, axis=1), output_shape=(lambda shape: (shape[0], shape[2])))
    
    x = Adder(x)
    print('x shape:',x.shape)
    
    encoded = Dense(1)(x)
    
    summer = Model(input_img, encoded)
    #adam = Adam(learning_rate=1e-3, epsilon=1e-3)
    #summer.compile(optimizer=adam, loss='mae')
    #summer.get_layer(index=1).set_weights([images])
    return summer

## Data prep

In [33]:
num_train_examples = 140000
num_val_examples = 25000
num_test_examples = 30000
max_train_length = 10
image_height = 28
image_width = 28
total_examples = num_train_examples + num_val_examples + num_test_examples
d = 10
k = 5
divisors = [1, 2, 5, 10]
#all_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
#all_indices = [0, 1, 2, 3]
all_indices = np.sort(np.random.choice(list(range(0, d)), k, replace=False)).tolist()
print('******True Indices******:',all_indices)
basepath = "./cyclic_discover_v1_Sk10_trails_1"

indices = all_indices
labels = all_labels[:,np.array(indices)]
labels = np.sum(labels,1)            

train_data = (features[0:num_train_examples], labels[0:num_train_examples])

val_data = (features[num_train_examples:num_train_examples+num_val_examples],
            labels[num_train_examples:num_train_examples+num_val_examples])

test_data = (features[num_train_examples+num_val_examples:],
            labels[num_train_examples+num_val_examples:])

******True Indices******: [1, 3, 4, 7, 9]


## Selction matrix

In [5]:
def new_matrix_all(k,d,indices,subgroup_indices):
    M3 = np.zeros((d,d))
    for row,index in zip(np.arange(k),indices):
        M3[row,index] = 1     

    if(k>1):
        if subgroup_indices == 0:                                
            M4 = np.eye(d*(d-1),d*(d-1))                                #L2, Sk     

        elif subgroup_indices == 1 or 2:    #D2k or Zk
            M4 = np.zeros((d*(d-1),d*(d-1)))
            M4[0:k-1,0:k-1] = np.eye(k-1)
            key_loc_Zk = ((d-k)*d) + k - 1
            M4[k-1, key_loc_Zk] = 1
            if subgroup_indices == 1: #D2k
                M4[k:(2*k)-1, -d+1:-d+k] = np.eye(k-1)
                key_loc_D2k = ((k-2)*d)
                M4[(2*k)-1, key_loc_D2k] = 1
    else:
        M4 =  np.zeros((d*(d-1),d*(d-1)))
        M4[0,0] = 1               

    return M3,M4

def generate_matrix_all(k_,subgroup_indices):
    #k_=int(np.random.choice(divisors,1,replace=False))
    train_indices=np.sort(np.random.choice(list(range(0,d)),k_,replace=False))
    #Generate matrix
    L1, L2 = new_matrix_all(len(train_indices),d,train_indices,subgroup_indices)
    #print('Matrix:',matrix)
    return L1, L2, train_indices

def generate_matrix_given_indices_all(train_indices,subgroup_indices):
    #Generate matrix
    L1, L2 = new_matrix_all(len(train_indices),d,train_indices,subgroup_indices)
    #print('Matrix:',matrix)
    return L1, L2   

## Checkpoints

In [38]:
filepath = 'saved_model.h5'
callback = tf.keras.callbacks.ModelCheckpoint(filepath,
                                                save_best_only=True,
                                                save_weights_only=True,)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, verbose=1, patience=patience, min_lr=0.000001)

def get_loss(M5,M6,epochs=5):
    model_discover.set_weights(initial_weights)
    #sample_output=Model_discover(val_ds)
    bias1 = np.zeros(d)
    bias2 = np.zeros(d*(d-1))
    model_discover.layers[2].set_weights([M5.T,bias1])
    model_discover.layers[6].set_weights([M6.T,bias2])

    start=time.time()
    train_history = model_discover.fit(train_data[0], 
                                        train_data[1], 
                                        epochs=epochs, 
                                        batch_size=128,
                                        shuffle=True,
                                        validation_data=val_data,
                                        callbacks=[callback,reduce_lr]) 
    
    end=time.time()
    print('Time:',end-start)
    
    return train_history

## Create and build model

In [42]:
try:
    del model_discover 
    tf.keras.backend.clear_session()
    gc.collect() 
except:
    print("model hasn't been yet defined")

model_discover = cyclic_model(d,0.01)
M1, M2 = get_matrix(d)

sample_y = np.random.randn(1, 10, 784)
sample_output = model_discover(sample_y)

x shape: (None, 30)


## Set trainable params

In [43]:
model_discover.layers[2].trainable = False
model_discover.layers[3].trainable = False
model_discover.layers[4].trainable = False
model_discover.layers[6].trainable = False

adam = Adam(learning_rate=1e-3, epsilon=1e-3)
model_discover.compile(optimizer=adam, loss='mae')

model_discover.layers[3].set_weights([M1.T])
model_discover.layers[4].set_weights([M2.T])

initial_weights = model_discover.get_weights()
model_discover.set_weights(initial_weights)

print(model_discover.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 784)]    0                                            
__________________________________________________________________________________________________
permute (Permute)               (None, 784, 10)      0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 784, 10)      110         permute[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 784, 90)      900         dense[0][0]                      
______________________________________________________________________________________________

## Contexts for bandit arms

In [ ]:
C = [list(combinations(range(d), i)) for i in range(1, d+1)]
C = [list(item) for sublist in C for item in sublist]

At = np.zeros((len(C), d))
for i, idx in zip(range(len(C)), C):
    At[i][idx] = 1

At = np.hstack((At, np.ones((At.shape[0], 1))))

C_d = [list(combinations(range(d), i)) for i in divisors]
C_d = [list(item) for sublist in C_d for item in sublist]

d_At = np.zeros((len(C_d), d))
for i, idx in zip(range(len(C_d)), C_d):
    d_At[i][idx] = 1

d_At = np.hstack((d_At, np.ones((d_At.shape[0], 1))))

s_arms = np.hstack((At, np.array([[1, 0, 0]] * At.shape[0])))
d_arms = np.hstack((d_At, np.array([[0, 1, 0]] * d_At.shape[0])))
z_arms = np.hstack((d_At, np.array([[0, 0, 1]] * d_At.shape[0])))

At = np.vstack((s_arms, d_arms, z_arms))
print('At shape:',At.shape)

## TS params

In [ ]:
#R = 0.01
R = 0.1
epsilon = 0.5
delta = 0.5
n_features = d+4
subgroup = {0:'Sk', 1:'D2k', 2:'Zk'}
B = np.eye(n_features)
B_inv = np.eye(n_features)
f = np.zeros((n_features, 1))
mu_hat = np.zeros((n_features, 1))
arm_iterations = 450
v = R * np.sqrt(24 / epsilon * n_features * np.log(1 / delta))
contexts = At
As = []
b = []

## Run Iterations

In [ ]:
for i in np.arange(arm_iterations):
    print("Iteration:",i)
    k_=int(np.random.choice(divisors,1,replace=False))
    subgroup_indices = 2 

    L1, L2, train_indices = generate_matrix(k_,subgroup_indices) 
    contexts_t = np.zeros(10)
    contexts_t[train_indices] = 1
    train_history = get_loss(L1,L2)
    min_loss = np.min(train_history.history['val_loss'])
    reward = -min_loss

    As.append(contexts_t)
    b.append(min_loss)
    print([min_loss,train_indices])

    np.save('c_r_Zk_As.npy',np.array(As))
    np.save('c_r_Zk_b.npy',np.array(b))


## Del model

In [ ]:
del model_discover 
tf.keras.backend.clear_session()
gc.collect()